In [14]:
import json

tmp = json.loads("""
{"/mnt/data/radgraph/data/fake_reports/report2.txt": {"text": "Lungs are clear . No focal acute infiltrate . No pleural effusion .", "entities": {"1": {"tokens": "Lungs", "label": "ANAT-DP", "start_ix": 0, "end_ix": 0, "relations": []}, "2": {"tokens": "clear", "label": "OBS-DP", "start_ix": 2, "end_ix": 2, "relations": [["located_at", "1"]]}, "3": {"tokens": "focal", "label": "OBS-DA", "start_ix": 5, "end_ix": 5, "relations": [["modify", "5"]]}, "4": {"tokens": "acute", "label": "OBS-DA", "start_ix": 6, "end_ix": 6, "relations": [["modify", "5"]]}, "5": {"tokens": "infiltrate", "label": "OBS-DA", "start_ix": 7, "end_ix": 7, "relations": []}, "6": {"tokens": "pleural", "label": "ANAT-DP", "start_ix": 10, "end_ix": 10, "relations": []}, "7": {"tokens": "effusion", "label": "OBS-DA", "start_ix": 11, "end_ix": 11, "relations": [["located_at", "6"]]}}, "data_source": null, "data_split": "inference"}, "/mnt/data/radgraph/data/fake_reports/report.txt": {"text": "The lungs are clear , without evidence of focal acute infiltrate or effusion .", "entities": {"1": {"tokens": "lungs", "label": "ANAT-DP", "start_ix": 1, "end_ix": 1, "relations": []}, "2": {"tokens": "clear", "label": "OBS-DP", "start_ix": 3, "end_ix": 3, "relations": [["located_at", "1"]]}, "3": {"tokens": "focal", "label": "OBS-DA", "start_ix": 8, "end_ix": 8, "relations": [["modify", "5"]]}, "4": {"tokens": "acute", "label": "OBS-DA", "start_ix": 9, "end_ix": 9, "relations": [["modify", "5"]]}, "5": {"tokens": "infiltrate", "label": "OBS-DA", "start_ix": 10, "end_ix": 10, "relations": []}, "6": {"tokens": "effusion", "label": "OBS-DA", "start_ix": 12, "end_ix": 12, "relations": []}}, "data_source": null, "data_split": "inference"}}
""")

In [15]:
tmp

{'/mnt/data/radgraph/data/fake_reports/report2.txt': {'text': 'Lungs are clear . No focal acute infiltrate . No pleural effusion .',
  'entities': {'1': {'tokens': 'Lungs',
    'label': 'ANAT-DP',
    'start_ix': 0,
    'end_ix': 0,
    'relations': []},
   '2': {'tokens': 'clear',
    'label': 'OBS-DP',
    'start_ix': 2,
    'end_ix': 2,
    'relations': [['located_at', '1']]},
   '3': {'tokens': 'focal',
    'label': 'OBS-DA',
    'start_ix': 5,
    'end_ix': 5,
    'relations': [['modify', '5']]},
   '4': {'tokens': 'acute',
    'label': 'OBS-DA',
    'start_ix': 6,
    'end_ix': 6,
    'relations': [['modify', '5']]},
   '5': {'tokens': 'infiltrate',
    'label': 'OBS-DA',
    'start_ix': 7,
    'end_ix': 7,
    'relations': []},
   '6': {'tokens': 'pleural',
    'label': 'ANAT-DP',
    'start_ix': 10,
    'end_ix': 10,
    'relations': []},
   '7': {'tokens': 'effusion',
    'label': 'OBS-DA',
    'start_ix': 11,
    'end_ix': 11,
    'relations': [['located_at', '6']]}},
  'data

In [13]:
from medvqa.utils.hashing import hash_string

def compute_label_set(data):
    entities = data['entities']
    n = len(entities)
    e_strings = [None] * n
    strings = []
    hashes = set()
    for k, e in entities.items():
        i = int(k)-1
        e_strings[i] = f"{e['tokens']}|{e['label']}" # tokens|label
        hashes.add(hash_string(e_strings[i]))
        strings.append(e_strings[i])
    for k, e in entities.items():
        i = int(k)-1
        for r in e['relations']:
            j = int(r[1])-1
            rel_s1 = f"{e_strings[i]}|{r[0]}|{e_strings[j]}" # e1|rel|e2
            rel_s2 = f"{e_strings[i]}|{e_strings[j]}" # e1|e2
            hashes.add(hash_string(rel_s1))
            hashes.add(hash_string(rel_s2))
            strings.append(rel_s1)
            strings.append(rel_s2)
    return strings, hashes

In [18]:
def print_entities_and_relations(data):
    print('Original text:')
    print(data['text'])
    print()
    entities = data['entities']
    n = len(entities)
    e_pairs = [None] * n
    for k, e in entities.items():
        e_pair = (e['tokens'], e['label'])
        e_pairs[int(k)-1] = e_pair
    relations = []
    for k, e in entities.items():
        i = int(k)-1
        for r in e['relations']:
            j = int(r[1])-1
            relations.append((e_pairs[i], r[0], e_pairs[j]))
    print('Entities:')
    for x in e_pairs:
        print(x)
    print()
    print('Relations:')
    for x in relations:
        print(x)

In [16]:
data = next(iter(tmp.values()))

In [19]:
print_entities_and_relations(data)

Original text:
Lungs are clear . No focal acute infiltrate . No pleural effusion .

Entities:
('Lungs', 'ANAT-DP')
('clear', 'OBS-DP')
('focal', 'OBS-DA')
('acute', 'OBS-DA')
('infiltrate', 'OBS-DA')
('pleural', 'ANAT-DP')
('effusion', 'OBS-DA')

Relations:
(('clear', 'OBS-DP'), 'located_at', ('Lungs', 'ANAT-DP'))
(('focal', 'OBS-DA'), 'modify', ('infiltrate', 'OBS-DA'))
(('acute', 'OBS-DA'), 'modify', ('infiltrate', 'OBS-DA'))
(('effusion', 'OBS-DA'), 'located_at', ('pleural', 'ANAT-DP'))


In [20]:
compute_label_set(data)

(['Lungs|ANAT-DP',
  'clear|OBS-DP',
  'focal|OBS-DA',
  'acute|OBS-DA',
  'infiltrate|OBS-DA',
  'pleural|ANAT-DP',
  'effusion|OBS-DA',
  'clear|OBS-DP|located_at|Lungs|ANAT-DP',
  'clear|OBS-DP|Lungs|ANAT-DP',
  'focal|OBS-DA|modify|infiltrate|OBS-DA',
  'focal|OBS-DA|infiltrate|OBS-DA',
  'acute|OBS-DA|modify|infiltrate|OBS-DA',
  'acute|OBS-DA|infiltrate|OBS-DA',
  'effusion|OBS-DA|located_at|pleural|ANAT-DP',
  'effusion|OBS-DA|pleural|ANAT-DP'],
 {(12, 309747879334862366),
  (12, 855426046901773212),
  (12, 2215373754232534800),
  (13, 1614911951124434649),
  (15, 19119047849380968),
  (15, 3677406714112377304),
  (17, 2836997461612593144),
  (26, 2987285217759247654),
  (30, 194462586615084465),
  (30, 3399811303483428276),
  (31, 2673881681973443813),
  (37, 408053797035404307),
  (37, 2527974376104740564),
  (37, 3180474612184124109),
  (42, 3185582518253405045)})

In [10]:
data['text'].split()

['The',
 'lungs',
 'are',
 'clear',
 ',',
 'without',
 'evidence',
 'of',
 'focal',
 'acute',
 'infiltrate',
 'or',
 'effusion',
 '.']

In [9]:

import subprocess

# --allennlp_bin_path /home/pamessina/miniconda3/envs/dygiepp/bin/allennlp \
# --digie_package_folder /home/pamessina/dygiepp/dygie/ \

subprocess.run("""conda run -n dygiepp python3 /home/pamessina/medvqa/medvqa/scripts/radgraph/radgraph_allennlp_inference.py \
--model_path /mnt/data/radgraph/data/models/model_checkpoint/model.tar.gz \
--dygie_package_parent_folder /home/pamessina/dygiepp/ \
--data_path /mnt/data/radgraph/data/fake_reports/ \
--out_path /home/pamessina/medvqa-workspace/tmp/radgraph/results.jsonl \
--temp_folder /home/pamessina/medvqa-workspace/tmp/radgraph/""", shell=True)
# subprocess.run('bash -c "conda activate dygiepp; python3 -V"', shell=True)

Getting paths to all the reports...
Got all the paths.
Preprocessing all the reports...
1 reports done
2 reports done
Done with preprocessing.
Running the inference now... This can take a bit of time
dygie_package_parent_folder: /home/pamessina/dygiepp/
Imported dygie successfully
/home/pamessina/dygiepp/dygie/__init__.py
Running command:  PYTHONPATH=/home/pamessina/dygiepp/:$PYTHONPATH conda run -n dygiepp allennlp predict /mnt/data/radgraph/data/models/model_checkpoint/model.tar.gz /home/pamessina/medvqa-workspace/tmp/radgraph/temp_dygie_input.json             --predictor dygie             --include-package dygie             --use-dataset-reader             --output-file /home/pamessina/medvqa-workspace/tmp/radgraph/temp_dygie_output.json             --cuda-device -1             --silent
Done with inference
Inference completed.
Postprocessing output file...
Done postprocessing.
Saving results and performing final cleanup...



2023-08-05 13:46:54,785 - INFO - allennlp.common.plugins - Plugin allennlp_models available
2023-08-05 13:46:54,804 - INFO - allennlp.models.archival - loading archive file /mnt/data/radgraph/data/models/model_checkpoint/model.tar.gz
2023-08-05 13:46:54,805 - INFO - allennlp.models.archival - extracting archive file /mnt/data/radgraph/data/models/model_checkpoint/model.tar.gz to temp dir /tmp/tmp6syt6e4e
2023-08-05 13:46:57,513 - INFO - allennlp.common.params - type = from_instances
2023-08-05 13:46:57,514 - INFO - allennlp.data.vocabulary - Loading token dictionary from /tmp/tmp6syt6e4e/vocabulary.
2023-08-05 13:46:57,514 - INFO - allennlp.common.params - model.type = dygie
2023-08-05 13:46:57,515 - INFO - allennlp.common.params - model.embedder.type = basic
2023-08-05 13:46:57,515 - INFO - allennlp.common.params - model.embedder.token_embedders.bert.type = pretrained_transformer_mismatched
2023-08-05 13:46:57,515 - INFO - allennlp.common.params - model.embedder.token_embedders.bert.m

CompletedProcess(args='conda run -n dygiepp python3 /home/pamessina/medvqa/medvqa/scripts/radgraph/radgraph_allennlp_inference.py --model_path /mnt/data/radgraph/data/models/model_checkpoint/model.tar.gz --dygie_package_parent_folder /home/pamessina/dygiepp/ --data_path /mnt/data/radgraph/data/fake_reports/ --out_path /home/pamessina/medvqa-workspace/tmp/radgraph/results.jsonl --temp_folder /home/pamessina/medvqa-workspace/tmp/radgraph/', returncode=0)

In [1]:
from medvqa.utils.files import load_jsonl
import json

In [15]:
!ls -lh /home/pamessina/medvqa-workspace/tmp/radgraph/results.json

-rw-rw-r-- 1 pamessina pamessina 1.7K Aug  5 11:26 /home/pamessina/medvqa-workspace/tmp/radgraph/results.json


In [10]:
tmp = load_jsonl('/home/pamessina/medvqa-workspace/tmp/radgraph/results.jsonl')

In [11]:
tmp

[{'/mnt/data/radgraph/data/fake_reports/report2.txt': {'text': 'Lungs are clear . No focal acute infiltrate . No pleural effusion .',
   'entities': {'1': {'tokens': 'Lungs',
     'label': 'ANAT-DP',
     'start_ix': 0,
     'end_ix': 0,
     'relations': []},
    '2': {'tokens': 'clear',
     'label': 'OBS-DP',
     'start_ix': 2,
     'end_ix': 2,
     'relations': [['located_at', '1']]},
    '3': {'tokens': 'focal',
     'label': 'OBS-DA',
     'start_ix': 5,
     'end_ix': 5,
     'relations': [['modify', '5']]},
    '4': {'tokens': 'acute',
     'label': 'OBS-DA',
     'start_ix': 6,
     'end_ix': 6,
     'relations': [['modify', '5']]},
    '5': {'tokens': 'infiltrate',
     'label': 'OBS-DA',
     'start_ix': 7,
     'end_ix': 7,
     'relations': []},
    '6': {'tokens': 'pleural',
     'label': 'ANAT-DP',
     'start_ix': 10,
     'end_ix': 10,
     'relations': []},
    '7': {'tokens': 'effusion',
     'label': 'OBS-DA',
     'start_ix': 11,
     'end_ix': 11,
     'relation

In [6]:
with open('/home/pamessina/medvqa-workspace/tmp/radgraph/results.json') as f:
    x = f.readline()
    print(json.loads(x))

{'/mnt/data/radgraph/data/fake_reports/report2.txt': {'text': 'Lungs are clear . No focal acute infiltrate . No pleural effusion .', 'entities': {'1': {'tokens': 'Lungs', 'label': 'ANAT-DP', 'start_ix': 0, 'end_ix': 0, 'relations': []}, '2': {'tokens': 'clear', 'label': 'OBS-DP', 'start_ix': 2, 'end_ix': 2, 'relations': [['located_at', '1']]}, '3': {'tokens': 'focal', 'label': 'OBS-DA', 'start_ix': 5, 'end_ix': 5, 'relations': [['modify', '5']]}, '4': {'tokens': 'acute', 'label': 'OBS-DA', 'start_ix': 6, 'end_ix': 6, 'relations': [['modify', '5']]}, '5': {'tokens': 'infiltrate', 'label': 'OBS-DA', 'start_ix': 7, 'end_ix': 7, 'relations': []}, '6': {'tokens': 'pleural', 'label': 'ANAT-DP', 'start_ix': 10, 'end_ix': 10, 'relations': []}, '7': {'tokens': 'effusion', 'label': 'OBS-DA', 'start_ix': 11, 'end_ix': 11, 'relations': [['located_at', '6']]}}, 'data_source': None, 'data_split': 'inference'}, '/mnt/data/radgraph/data/fake_reports/report.txt': {'text': 'The lungs are clear , without

In [5]:
import importlib

importlib.import_module('dygie')

ModuleNotFoundError: No module named 'dygie'

In [1]:
!ls /home/pamessina/dygiepp/dygie

data	     models	 __pycache__  spacy_interface  training
__init__.py  predictors  pytest.ini   tests


In [3]:
import importlib
import sys

# Add the path to the folder where 'digie' module is defined
module_path = '/home/pamessina/dygiepp'
sys.path.append(module_path)

# Now import the module
try:
    importlib.import_module('dygie')
except ModuleNotFoundError:
    print("Module 'digie' not found. Please check if the path is correct.")

In [6]:
import dygie

In [7]:
from dygie import 

<module 'dygie' from '/home/pamessina/dygiepp/dygie/__init__.py'>